In [321]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
import matplotlib
%matplotlib inline

In [322]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from example.census import clean_census_data
features, output = clean_census_data()

In [2]:
features[:10]

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass- ?,workclass- Federal-gov,workclass- Local-gov,workclass- Never-worked,workclass- Private,...,native-country- Portugal,native-country- Puerto-Rico,native-country- Scotland,native-country- South,native-country- Taiwan,native-country- Thailand,native-country- Trinadad&Tobago,native-country- United-States,native-country- Vietnam,native-country- Yugoslavia
0,39,13,2174,0,40,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,13,0,0,13,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,9,0,0,40,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,53,7,0,0,40,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,28,13,0,0,40,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,37,14,0,0,40,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
6,49,5,0,0,16,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,52,9,0,0,45,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,31,14,14084,0,50,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
9,42,13,5178,0,40,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [7]:
DATA_TO_USE, _ = train_test_split(df, train_size=0.7, random_state=42)

GET THE FEATURE NAMES

In [11]:
cls = RandomForestClassifier(min_samples_leaf = 10, random_state=42, max_depth=20, n_estimators=40)

In [12]:
DATA_TO_USE, _, OUTPUT_TO_USE, _ = train_test_split(features, output, random_state=42, train_size=0.7)

In [13]:
TRAIN_FEATURES, TEST_FEATURES, TRAIN_OUTPUT, TEST_OUTPUT = train_test_split(DATA_TO_USE, OUTPUT_TO_USE, random_state=42, train_size=0.7)

In [14]:
cls.fit(TRAIN_FEATURES, TRAIN_OUTPUT)
predicted = cls.predict(TEST_FEATURES)
predicted_probs = cls.predict_proba(TEST_FEATURES).transpose()[1]
predicted = pd.Series(predicted, index=TEST_OUTPUT.index)
predicted.name = 'predicted'

In [15]:
from sklearn.metrics import precision_score, recall_score

In [16]:
print precision_score(TEST_OUTPUT, predicted)
print recall_score(TEST_OUTPUT, predicted)

0.794960903562
0.560318432333


In [288]:
misclassified_indices = TEST_OUTPUT[(TEST_OUTPUT != predicted)].index
misclassified_features = TEST_FEATURES[np.nonzero((TEST_OUTPUT != predicted).values)]
misclassified_features = np.array(misclassified_features)

In [273]:
(TEST_OUTPUT != predicted).index

Int64Index([28626, 22861, 24015, 14057, 13317, 17290, 20528, 25528,  3311,
            17156, 
            ...
            30734, 10069, 26851, 13183,  7637,  3029,  2648, 16646,  7655,
            16330],
           dtype='int64', length=6838)

In [23]:
misclassified_df = df.ix[misclassified_indices].join(predicted)

In [24]:
def distribution(variable):
    print "whole dataset"
    print df[variable].value_counts() * 100.0 / len(df[variable])
    print "\nmisclassified"
    print misclassified_df[variable].value_counts() * 100.0 / len(misclassified_df[variable])

In [47]:
import numba
def distribution(instance):
    preds = np.zeros(len(cls.estimators_))
    for i, est in enumerate(cls.estimators_):
        preds[i] = est.predict(instance)[0]
    return pd.Series(preds).value_counts()

In [87]:
%%timeit
cls.predict(misclassified_features[0])

100 loops, best of 3: 2.23 ms per loop


In [49]:
distribution(misclassified_features[0])

1    38
0     2
dtype: int64

In [50]:
cols = [x for x in df.columns if x != 'fnlwgt']

In [51]:
to_group = df
to_group['age'] = np.floor((to_group['age']/ 5)) * 5
counted = to_group.groupby(cols).size().reset_index()
counted[counted[0] > 2]

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary,0
12,15,?,10th,6,Never-married,?,Own-child,White,Female,0,0,20,United-States,<=50K,3
19,15,?,10th,6,Never-married,?,Own-child,White,Male,0,0,30,United-States,<=50K,5
20,15,?,10th,6,Never-married,?,Own-child,White,Male,0,0,40,United-States,<=50K,11
40,15,?,11th,7,Never-married,?,Own-child,White,Female,0,0,20,United-States,<=50K,3
42,15,?,11th,7,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K,3
51,15,?,11th,7,Never-married,?,Own-child,White,Male,0,0,25,United-States,<=50K,4
55,15,?,11th,7,Never-married,?,Own-child,White,Male,0,0,40,United-States,<=50K,5
98,15,?,HS-grad,9,Never-married,?,Own-child,White,Female,0,0,20,United-States,<=50K,3
110,15,?,HS-grad,9,Never-married,?,Own-child,White,Male,0,0,30,United-States,<=50K,4
113,15,?,HS-grad,9,Never-married,?,Own-child,White,Male,0,0,40,United-States,<=50K,3


In [52]:
cat = df[(df.age >= 30) & (df['hours-per-week'] >= 40) & (df.occupation == ' Exec-managerial') & (df.education == ' HS-grad') & (df.workclass == ' Private') & (df.relationship == ' Husband') & (df.race == ' White')]
print cat.salary.value_counts()
cat.sort('age', ascending=True)

 >50K     100
 <=50K     73
dtype: int64


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
27589,30,Private,225053,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,<=50K
26368,30,Private,381153,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
26087,30,Private,167990,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,Ireland,<=50K
8814,30,Private,159255,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,60,United-States,>50K
27447,30,Private,344200,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
4265,30,Private,34104,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,3103,0,55,United-States,>50K
7672,30,Private,99309,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,<=50K
4143,30,Private,223327,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
4095,30,Private,45604,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,54,United-States,<=50K
24792,30,Private,243165,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,<=50K


In [215]:
import forestspy
spy = forestspy.ForestSpy(cls, feature_names)

# predict node ids

In [188]:
for _ in range(1000):

    spy.predict_nodes(misclassified_fbeatures[0])

NameError: name 'misclassified_fbeatures' is not defined

In [214]:
%%timeit
spy.predict_nodes(misclassified_features[0])


10000 loops, best of 3: 61.7 µs per loop


In [262]:
import forestspy
spy = forestspy.ForestSpy(cls, feature_names)
spy.predict_nodes(misclassified_features[8])


[91,
 132,
 324,
 84,
 149,
 124,
 251,
 214,
 200,
 290,
 377,
 182,
 322,
 179,
 26,
 312,
 252,
 195,
 172,
 160,
 228,
 103,
 187,
 272,
 233,
 162,
 293,
 344,
 73,
 107,
 133,
 260,
 159,
 141,
 100,
 132,
 167,
 240,
 327,
 152]

In [250]:
import forestspy
spy = forestspy.ForestSpy(cls, feature_names)
%lprun -f spy.predict_nodes -f spy.trees[0].node -f spy.trees[0].predicted_node spy.predict_nodes(misclassified_features[0])

In [251]:
def f():
    for _ in xrange(1000):
        spy.predict_nodes(misclassified_features[0])

In [253]:
%lprun -f spy.predict_nodes -f spy.trees[0].node -f spy.trees[0].predicted_node f()
        

In [310]:
misclassified_node_ids = pd.DataFrame((spy.predict_nodes(f) for f in np.array(misclassified_features)))
train_node_ids = pd.DataFrame((spy.predict_nodes(f) for f in np.array(TRAIN_FEATURES)))
test_node_ids = pd.DataFrame((spy.predict_nodes(f) for f in np.array(TEST_FEATURES)))

In [305]:
misclassified_node_ids[1].value_counts()[:5]

132    337
236    115
218     33
238     25
258     22
dtype: int64

In [306]:
train_node_ids[1].value_counts()[:5]

132    2443
167     986
236     870
218     786
159     624
dtype: int64

In [309]:
df = test_df[test_node_ids[1] == 258]
nontrivial_columns = df.columns[(df[:5] != 0).any()]
df[nontrivial_columns]

,age,education-num,capital-gain,hours-per-week,workclass- Federal-gov,workclass- Private,workclass- Self-emp-not-inc,marital-status- Divorced,marital-status- Never-married,occupation- Adm-clerical,occupation- Exec-managerial,occupation- Farming-fishing,occupation- Sales,relationship- Not-in-family,race- White,sex- Male,native-country- United-States,salary
63,34,13,0,45,0,1,0,0,1,0,0,0,1,1,1,1,1,0
266,54,13,27828,50,0,0,1,1,0,0,0,1,0,1,1,1,1,1
355,50,13,0,50,0,1,0,0,1,0,1,0,0,1,1,1,1,1
714,45,13,0,55,1,0,0,0,1,1,0,0,0,1,1,1,1,1
824,43,13,0,60,0,1,0,0,1,0,0,0,1,1,1,1,1,1
845,50,13,0,50,0,1,0,1,0,0,1,0,0,1,1,0,1,1
1119,41,13,0,50,0,1,0,1,0,0,1,0,0,1,1,1,1,1
1157,35,13,8614,45,0,1,0,0,1,0,1,0,0,1,1,1,1,1
1165,43,13,0,50,0,1,0,1,0,0,0,0,1,1,1,0,1,0
1418,48,13,0,50,0,1,0,0,1,0,1,0,0,1,1,1,1,0


In [295]:
node_ids[0].value_counts()[:5]

91     254
231     62
135     59
108     41
298     36
dtype: int64

In [269]:
node_ids[4].value_counts()

402    5884
149     954
dtype: int64

In [81]:
test_df = pd.DataFrame(TEST_FEATURES, columns=feature_names, index=TEST_OUTPUT.index)
test_df['salary'] = TEST_OUTPUT
test_df.index = range(len(test_df))

In [217]:
len(node_ids)

15954

In [243]:
nids = node_ids[:2000].values
@numba.autojit(nopython=True)
def f(nids):
    arr = np.zeros(shape=(len(nids), len(nids)))
    for i in xrange(len(nids)):
        for j in xrange(len(nids)):
            for node_idx in range(40):
                if nids[i][node_idx] == nids[j][node_idx]:
                    arr[i][j] += 1
    return arr


In [244]:
%%timeit
f(node_ids[:100].values)

The slowest run took 224.93 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 733 µs per loop


In [223]:
node_ids.values[i]

array([[228, 159, 123, ..., 269,  16, 143],
       [151, 238, 401, ..., 213, 343, 152],
       [163,  25, 288, ..., 199, 250, 152],
       ..., 
       [187, 186,  28, ...,  22,  55,  18],
       [192, 127, 275, ..., 214, 131, 152],
       [ 98,  26, 419, ..., 205, 333, 152]])

In [216]:
distances = pd.DataFrame(index=node_ids.index)
for i in node_ids.index:
    distances[i] = (node_ids != node_ids.ix[i]).astype(int).sum(axis=1)

KeyboardInterrupt: 

In [62]:
len(distances)

6838

In [63]:
def smush_cliques(cliques, ids):
    ids = set(ids)
    disjoint_cliques = [ids]
    for c in cliques:
        if c.intersection(ids):
            ids.update(c)
        else:
            disjoint_cliques.append(c)
    return disjoint_cliques
cliques = list()
for i in distances.columns:
    near_ids = set(distances[i][distances[i] < 10].index)
    cliques = smush_cliques(cliques, near_ids)
cliques = list(reversed(sorted(cliques, key=len)))

In [134]:
b

1

In [132]:
a, b = df.ix[0]

In [130]:
df = pd.DataFrame({'s': [1,2,3], 'a': [2,3,3]})

In [64]:
def similarity_df(clique_id):
    indices = list(cliques[clique_id])
    return similarity_df_indices(indices)
def similarity_df_indices(indices):
    similar_df = pd.DataFrame(my_test_features[indices], columns=feature_names, index=indices)
    similar_df['pred'] = np.array(predicted)[similar_df.index]
    similar_df['pred_prob'] = np.array(predicted_probs)[similar_df.index]
    similar_df['actual'] = np.array(TEST_OUTPUT)[similar_df.index]
    return similar_df
def filtered_similarity_df_indices(indices):    
    similar_df = similarity_df_indices(indices)
    nontrivial_columns = similar_df.columns[(similar_df != 0).any()]
    similar_df = similar_df[nontrivial_columns]
    similar_df['pred'] = np.array(predicted)[similar_df.index]
    similar_df['pred_prob'] = np.array(predicted_probs)[similar_df.index]
    similar_df['actual'] = np.array(TEST_OUTPUT)[similar_df.index]
    return similar_df
def filtered_similarity_df(clique_id):
    indices = list(cliques[clique_id])
    return filtered_similarity_df_indices(indices)
#similarity_df(1)

In [65]:
def get_prediction_accuracy(clique_index):
    df = similarity_df(clique_index)
    pct_wrong = np.count_nonzero(df.actual != df.pred) * 100.0 / len(df)
    pct_rich = np.sum(df.pred) * 100.0 / len(df)
    pct_exec = np.count_nonzero(df['occupation- Exec-managerial'] == 1) * 100.0 / len(df)
    pct_sales = np.count_nonzero(df['occupation- Sales'] == 1) * 100.0 / len(df)
    return pct_wrong, len(df), pct_rich, pct_exec, pct_sales

In [66]:
accuracy_df = pd.DataFrame((get_prediction_accuracy(i) for i in range(len(cliques[:100]))), 
                           columns=['pct_wrong', 'size', 'pct_rich', 'pct_exec', 'pct_sales'])

In [ ]:
%%

In [84]:
node_ids[0].value_counts()[:10]

91     746
11     421
108    214
182    191
135    186
231    169
151    150
61     149
318    140
79     138
dtype: int64

In [85]:
indices = node_ids[0] == 11
test_df[indices]

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass- ?,workclass- Federal-gov,workclass- Local-gov,workclass- Never-worked,workclass- Private,...,native-country- Puerto-Rico,native-country- Scotland,native-country- South,native-country- Taiwan,native-country- Thailand,native-country- Trinadad&Tobago,native-country- United-States,native-country- Vietnam,native-country- Yugoslavia,salary
41,19,9,2176,0,45,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
44,23,9,0,0,40,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
53,22,10,0,0,30,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
66,21,9,0,0,20,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
72,18,10,0,0,20,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
117,17,8,0,0,12,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
125,19,6,0,0,25,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
127,23,9,0,0,40,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
179,19,9,0,0,50,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
190,23,9,0,0,40,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [67]:
accuracy_df

,pct_wrong,size,pct_rich,pct_exec,pct_sales
0,33.568905,283,0.000000,0,0.000000
1,0.523560,191,0.000000,0,15.706806
2,0.000000,153,0.000000,0,45.098039
3,0.000000,100,0.000000,0,0.000000
4,24.444444,90,97.777778,0,0.000000
5,15.714286,70,0.000000,0,0.000000
6,0.000000,69,0.000000,0,0.000000
7,30.882353,68,7.352941,0,100.000000
8,12.068966,58,100.000000,100,0.000000
9,43.181818,44,54.545455,100,0.000000


In [887]:
s = pd.Series(np.array(predicted) != np.array(TEST_OUTPUT))
misclassified_df = pd.DataFrame(s, columns=['misclassified'])
clique_indices = pd.Series(index=misclassified_df.index)
for i, clique in enumerate(cliques):
    for member in clique:
        clique_indices[member] = i
misclassified_df['clique'] = clique_indices

In [898]:
misclassified_df[misclassified_df.misclassified]

,misclassified,clique
1,True,137
3,True,730
4,True,24
7,True,731
8,True,4
9,True,0
13,True,27
14,True,733
15,True,98
16,True,55


In [904]:
i = 6751
filtered_similarity_df_indices(list(distances[i][distances[i] < 30].index))

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass- ?,workclass- Private,workclass- Self-emp-not-inc,workclass- State-gov,marital-status- Married-civ-spouse,...,race- White,sex- Female,sex- Male,native-country- Dominican-Republic,native-country- England,native-country- South,native-country- United-States,pred,pred_prob,actual
496,31,13,0,1902,40,0,0,1,0,1,...,1,1,0,0,0,0,1,1,0.677906,1
595,37,13,0,0,37,0,1,0,0,1,...,1,1,0,0,0,0,1,1,0.597780,1
718,33,13,0,0,40,0,1,0,0,1,...,1,0,1,0,0,0,1,1,0.660854,1
1674,27,13,0,0,45,0,1,0,0,1,...,0,1,0,0,0,0,1,1,0.526672,1
1756,33,13,0,0,40,0,1,0,0,1,...,1,0,1,0,0,0,1,1,0.660854,1
1817,26,13,0,0,40,0,0,0,1,1,...,0,0,1,0,0,0,1,0,0.469448,1
2028,57,13,0,0,40,0,1,0,0,1,...,1,1,0,0,0,0,1,1,0.599278,0
2167,27,10,0,0,40,0,1,0,0,1,...,1,1,0,0,0,0,1,0,0.433253,1
2220,30,13,0,0,50,0,1,0,0,1,...,1,0,1,0,0,0,1,1,0.626121,1
2481,27,13,0,0,40,0,1,0,0,1,...,1,0,1,1,0,0,0,1,0.500186,0


In [863]:
accuracy_df[:20]

,pct_wrong,size,pct_rich,pct_exec,pct_sales
0,33.568905,283,0.000000,0,0.000000
1,0.523560,191,0.000000,0,15.706806
2,0.000000,153,0.000000,0,45.098039
3,0.000000,100,0.000000,0,0.000000
4,24.444444,90,97.777778,0,0.000000
5,15.714286,70,0.000000,0,0.000000
6,0.000000,69,0.000000,0,0.000000
7,30.882353,68,7.352941,0,100.000000
8,12.068966,58,100.000000,100,0.000000
9,43.181818,44,54.545455,100,0.000000


In [864]:
filtered_similarity_df(0)

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass- Private,marital-status- Married-civ-spouse,occupation- Craft-repair,relationship- Husband,race- White,sex- Male,native-country- Mexico,native-country- United-States,pred_prob,actual,pred
6146,28,9,0,0,50,1,1,1,1,1,1,0,1,0.256721,0,0
9,49,9,0,0,40,1,1,1,1,1,1,0,1,0.351920,1,0
4108,42,9,0,0,50,1,1,1,1,1,1,0,1,0.400345,1,0
1037,41,9,0,0,40,1,1,1,1,1,1,0,1,0.322436,1,0
1197,26,10,2829,0,40,1,1,1,1,1,1,0,1,0.219756,0,0
1044,30,9,0,0,40,1,1,1,1,1,1,0,1,0.189404,1,0
2072,53,10,0,0,36,1,1,1,1,1,1,0,1,0.302259,1,0
31,54,9,0,0,40,1,1,1,1,1,1,0,1,0.351920,1,0
1058,27,9,0,0,45,1,1,1,1,1,1,0,1,0.235523,1,0
3111,46,10,3103,0,60,1,1,1,1,1,1,0,1,0.400234,1,0


In [850]:
accuracy_df[accuracy_df.pct_wrong > 20].sort('pct_wrong', ascending=False)

,pct_wrong,size,pct_rich,pct_exec,pct_sales
37,100.000000,4,0.000000,0.000000,0.000000
23,77.777778,9,0.000000,100.000000,0.000000
33,50.000000,4,0.000000,0.000000,0.000000
39,50.000000,4,100.000000,0.000000,0.000000
17,34.782609,23,13.043478,0.000000,0.000000
28,33.333333,6,83.333333,0.000000,0.000000
3,30.739300,257,82.101167,0.000000,0.000000
4,30.454545,220,12.727273,17.272727,9.545455
2,27.415144,383,78.590078,100.000000,0.000000
1,25.670945,1714,12.660443,0.000000,18.203034


In [815]:
filtered_similarity_df(0).describe()

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass- Federal-gov,workclass- Local-gov,workclass- Private,workclass- Self-emp-inc,workclass- Self-emp-not-inc,...,native-country- Canada,native-country- Ecuador,native-country- Germany,native-country- Jamaica,native-country- Mexico,native-country- United-States,native-country- Vietnam,pred,pred_prob,actual
count,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,...,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000
mean,42.765766,9.639640,217.518018,68.432432,43.972973,0.013514,0.040541,0.779279,0.040541,0.112613,...,0.004505,0.004505,0.004505,0.009009,0.004505,0.963964,0.004505,0.121622,0.336892,0.306306
std,11.906844,2.057072,857.793886,334.145621,11.000172,0.115720,0.197669,0.415670,0.197669,0.316833,...,0.067116,0.067116,0.067116,0.094701,0.067116,0.186801,0.067116,0.327587,0.122331,0.462000
min,21.000000,2.000000,0.000000,0.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.134539,0.000000
25%,33.000000,9.000000,0.000000,0.000000,40.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.238942,0.000000
50%,42.000000,9.000000,0.000000,0.000000,40.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.319196,0.000000
75%,53.000000,10.000000,0.000000,0.000000,50.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.407099,1.000000
max,79.000000,14.000000,5013.000000,1887.000000,99.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.629803,1.000000


In [751]:
accuracy_df

,pct_wrong,size,pct_rich
0,3.233256,433,0.692841
1,27.821522,381,35.170604
2,29.538462,325,22.769231
3,12.500000,112,6.250000
4,3.092784,97,0.000000
5,35.869565,92,40.217391
6,0.000000,86,0.000000
7,25.925926,54,81.481481
8,0.000000,42,0.000000
9,7.894737,38,0.000000


In [695]:
get_prediction_accuracy(1)

/home/bork/.conda/lib/python2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0